<h1>Imports</h1>
<p>This is importing the 3 libraries we need.</p>
<p>The first two (librosa and soundfile) allows us to edit and save audio files.</p>
<p>The next one (numpy) supplies a load of numerical functions.</p>

In [12]:
import librosa
import soundfile as sf
import numpy
import requests
import re
import json

In [30]:
url = "https://www.globalplayer.com/catchup/lbc/uk/46vyD81/"
res = requests.get(url)
text = res.text
text_res = re.search('__NEXT_DATA__" type="application/json">(.+?)</script>', text)
props_str = text_res.group(1)
json_props = json.loads(props_str)
streamUrl = json_props['props']['pageProps']['catchupInfo']['episodes'][0]['streamUrl']

http://catchup.hosting.thisisdax.com/045981f8-79a9-4036-813e-56da49b0fd5f/29f426f6-53ba-42db-a7fb-738c68d686a9.m4a?aw_0_1st.channelid=d2bf085f-d4e0-4a6b-80d6-75d92bf0fcae&aw_0_1st.showid=045981f8-79a9-4036-813e-56da49b0fd5f&aw_0_1st.episodeid=51598366-b95e-4001-b71d-58f19d3a5ee8


In [28]:
streamRes = requests.get(streamUrl)
open('originalstream.m4a', 'wb').write(streamRes.content)

64161769

<h1>Loading the main audio</h1>
<p>Firstly, we will use the librosa load function to load the 3 hour LBC show (James O'Brien of course).</p>
<p>We save this loaded file to two variables. The first variable called 'clip' holds the audio file as an array of numbers.</p>
<p>The second variable called 'sr' holds the soundrate as a number. This is irrelevant and useless until we save the file after editing it as we need the original soundrate. (Don't worry about this)</p>

In [31]:
clip, sr = librosa.load('originalstream.m4a')

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


FileNotFoundError: [Errno 2] No such file or directory: 'originalstream.m4a'

<h2>Loading the audio to match with the show</h2>
<p>I've previously clipped down an LBC show to the jingle they play immediately before and after the adverts. Depending on the time of day, they play one of a selection of a few jingles loaded below before going to an advert break and before coming back.</p>
<p>I've used the librosa load function and named these clips appropriately as 'start_matcher' or 'news_matcher'. I've used an underscore (_) for the second variable (which means discard) as the soundrate isn't important for these files as we won't be saving them.</p>

In [3]:
start_matcher, _ = librosa.load('ad_start.mp3')

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


In [4]:
news_matcher, _ = librosa.load('news_start.mp3')

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


In [5]:
end_matcher, _ = librosa.load('ad_end.mp3')

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


In [6]:
alt_matcher, _ = librosa.load('alt_ads.mp3')

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


In [7]:
alt2_matcher, _ = librosa.load('alt2_ads.mp3')

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


In [8]:
alt3_matcher, _ = librosa.load('alt3_ads.mp3')

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


In [9]:
alt4_matcher, _ = librosa.load('alt4_ads.mp3')

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


In [10]:
alt5_matcher, _ = librosa.load('alt5_ads.mp3')

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


<h1>Get matching scores function</h1>
<p>Below, I've defined the 'matching scores' function. This takes two inputs and outputs two values. On the first line, it calls the librosa frame utility function to split the clip input (3 hour LBC show) into sections the length of the matcher (the LBC adverts intro or outro - usually about 5 seconds). It saves these 5 second snippets of the 3 hour show as an array into a variable called 'frames'</p>
<p>We then create an empty array called 'scores'</p>
<p>We loop through the array we just created called 'frames' containing 5 second snippets. For every 5 second snippet we use numpy to compare the 5 second snippet and the matcher (LBC adverts intro or outro). If they are similar, we know LBC is about to go to an adverts break or come back from an adverts break. If they are similar, numpy generates a high score, often over 200, however if they are different, numpy generates a lower score, often under 200.</p>
<p>We then add the score to the empty array of scores we previously created.</p>
<p>When this loop is complete, we will have an array of scores corresponding to the array of 5 second frames with every score describing the similarity between the frame and the preset intro/outro clip.</p>
<p>We 'return' this array of scores along with the array of frames.</p>

In [11]:
def get_scores(clip, matcher):
    frames = librosa.util.frame(clip, len(matcher), 45, axis=0)
    scores = []
    for x, frame in enumerate(frames):
        num = numpy.correlate(frame, matcher)
        scores.append(num[0])
    return scores, frames

<h2>Calling the 'get matching scores' function</h2>
<p>We now call the function above with the two parameters. Firstly, we pass in the 3 hour clip we imported using librosa load above. Secondly, we pass in the matcher to generate the correlation array, telling us how similar every 5 second clip within the 3 hour show and the 5 second advert intro or outro are.</p>
<p>We do this for every intro/outro matcher imported above using the librosa load function.</p>

In [12]:
start_scores, start_frames = get_scores(clip, start_matcher)
print('start done')
end_scores, end_frames = get_scores(clip, end_matcher)
print('end done')

start done
end done


In [13]:
news_scores, news_frames = get_scores(clip, news_matcher)
print('news done')

news done


In [14]:
alt_scores, alt_frames = get_scores(clip, alt_matcher)

In [15]:
alt2_scores, alt2_frames = get_scores(clip, alt2_matcher)

In [16]:
alt3_scores, alt3_frames = get_scores(clip, alt3_matcher)

In [17]:
alt4_scores, alt4_frames = get_scores(clip, alt4_matcher)

In [18]:
alt5_scores, alt5_frames = get_scores(clip, alt5_matcher)

<h1>Getting the significant times</h1>
<p>The 'get times' function looks through the scores array generated by the 'get matching scores' function and returns the scores above a certain threshold along with their timestamp so we know exactly where the adverts start and finish playing in the 3 hour show.</p>
<p>It takes in 4 parameters. Scores: the previously generated array of scores. Matcher: the 5(ish) second clip we're searching for. Threshold: a value often around 200 which the correlation score must be above for us to assume the clips are the same (and assume we have found the location of the adverts intro/outro). Frames - the original array of the 3 hour show split into 5 second clips.</p>

<p>We create two empty arrays to start with. 'done_times' and 'ret_times'. We then loop through the previously generated array of scores (given as an input) backwards.</p>
<p>'done_times' will hold any times that we've marked as intro/outro (where the ads begin) to the nearest minute. This will later ensure that we don't mark any minutes multiple times. For example, if the intro is playing over 5 seconds and we check every individual second within that time, we could mark that timestamp 5 times which would be unneccesary.</p>
<p>'ret_times' will hold any times that we've marked as intro/outro, similarly to 'done_times', but to a much more precise value. We will return 'ret_times' and use this to cut out the adverts as it is to the nearest second not the nearest minute.</p>

<p>Firstly, we check that the current value is greater than the threshold we have set. If it is less, for example the threshold is 200 and the correlation between the current 5 second clip and the advert intro/outro is only 150, we discard this value and move on. Assuming it is greater than the threshold, we continue with this value.</p>
<p>We calculate how far through the 3 hour show this value is found using a short formula as a decimal where 0 is the start and 1 is the end (3 hours in).</p>
<p>We then use a short function to calculate the number of hours, minutes and seconds into the show and save into hours, mins and secs variables respectively.</p>
<p>We check if the value is in 'done_times', meaning we have already marked it using the hours and minutes. If it is, we 'continue' to the next value and return to the start of the loop for the next value.</p>

<p>Assuming the value isn't in 'done_times', we keep going and add the value to 'done_times' so we don't duplicate it later. We also add the accurate value to 'ret_times' to help us edit the 3 hour show. We print the current index and value for debugging purposes as well as the hours and minutes to the user. We save the short clip to a file to double check it matches the intro/outro</p>

<p>After the loop is complete, we return ret_times which contains all the timestamps in the show where the adverts intro/outro is playing.</p>

In [19]:
def calc_val(way_through):
    hours = numpy.floor(way_through * 3)
    mins = ((way_through * 3) - hours)*60
    secs = (((way_through * 3) - hours)*60 - mins ) * 60
    return hours, mins, secs

def get_times(scores, matcher, threshold, frames):
    done_times = []
    ret_times = []
    for x, num in reversed(list(enumerate(scores))):
        if num > threshold:
            way_through = x/(len(scores)-(len(matcher)/45))
            hours, mins, secs = calc_val(way_through)
            if [hours, numpy.floor(mins)] in done_times:
                continue
            done_times.append([hours, numpy.floor(mins)])
            ret_times.append([way_through, x])
            print(x)
            print(num)
            print('It is ' + str(hours) + ' hours and ' + str(mins) + ' mins and ' + str(secs) + ' secs.')
            sf.write(str(num)+'.wav', frames[x], sr)
    return ret_times

<h2>Calling the get times function</h2>
<p>We call the get times function with the scores, matcher, threshold and frames inputs.</p>
<p>We want the threshold to be as high as possible to avoid any false positives but not too high or we could miss some advert intros/outros that sound slightly different.</p>
<p>I have selected appropriate values for all the clips depending on how regularly they are played.</p>

In [20]:
start_times = get_times(start_scores, start_matcher, 200, start_frames)

2703862
262.998
It is 1.0 hours and 32.00239620705149 mins and 0.0 secs.


In [21]:
end_times = get_times(end_scores, end_matcher, 300, end_frames)

4078454
1443.5284
It is 2.0 hours and 18.877574744514256 mins and 0.0 secs.
3242122
1871.8562
It is 1.0 hours and 50.39919547599018 mins and 0.0 secs.
2376723
338.36176
It is 1.0 hours and 20.931040555932753 mins and 0.0 secs.
1458297
1042.5073
It is 0.0 hours and 49.65723546647845 mins and 0.0 secs.
654971
2073.5515
It is 0.0 hours and 22.30276080298791 mins and 0.0 secs.


In [22]:
news_times = get_times(news_scores, news_matcher, 400, news_frames)

3560208
431.57855
It is 2.0 hours and 1.4315559984279602 mins and 0.0 secs.
1786055
520.3011
It is 1.0 hours and 0.9187546763482057 mins and 0.0 secs.
13791
472.25644
It is 0.0 hours and 0.47038335647083546 mins and 0.0 secs.


In [23]:
alt_times = get_times(alt_scores, alt_matcher, 150, alt_frames)

2703982
247.05615
It is 1.0 hours and 32.00064166993466 mins and 0.0 secs.


In [24]:
alt2_times = get_times(alt2_scores, alt2_matcher, 200, alt2_frames)

4440982
243.55203
It is 2.0 hours and 31.110329000228365 mins and 0.0 secs.


In [25]:
alt3_times = get_times(alt3_scores, alt3_matcher, 400, alt3_frames)

4903246
426.77112
It is 2.0 hours and 47.00476767430484 mins and 0.0 secs.
4530644
423.14563
It is 2.0 hours and 34.31392767872207 mins and 0.0 secs.
3691194
446.57178
It is 2.0 hours and 5.722224911984419 mins and 0.0 secs.
2793445
475.41016
It is 1.0 hours and 35.144855721281054 mins and 0.0 secs.
2376188
410.54956
It is 1.0 hours and 20.933064523067173 mins and 0.0 secs.
1896823
476.55872
It is 1.0 hours and 4.605872198596179 mins and 0.0 secs.
1065519
449.20288
It is 0.0 hours and 36.29162253893801 mins and 0.0 secs.
113871
403.1865
It is 0.0 hours and 3.8784511117412364 mins and 0.0 secs.


In [26]:
alt4_times = get_times(alt4_scores, alt4_matcher, 200, alt4_frames)

In [27]:
alt5_times = get_times(alt5_scores, alt5_matcher, 200, alt5_frames)

<h1>Combining the times arrays</h1>
<p>Now we have 8 arrays with the timestamps (location within audio) for when the adverts begin/end on LBC.</p>
<p>Below we combine them into one big array with all the timestamps for all the various intros/outros.</p>

In [28]:
print(alt_times)
fin_times_zipped_arr = [*alt_times, *alt2_times, *alt3_times, *alt4_times, *alt5_times, *start_times, *news_times, *end_times]

[[0.5111146759440814, 2703982]]


In [29]:
fin_times_arr, fin_timestamps_arr = zip(*fin_times_zipped_arr)
fin_timestamps_arr = sorted(fin_timestamps_arr)
sorted_times_arr = sorted(fin_times_arr)
print(sorted_times_arr)

[0.0026132408692824192, 0.021546950620784645, 0.12390422668326616, 0.20162012521632225, 0.2758735303693247, 0.3384375259797122, 0.35892151221442326, 0.4496168919774042, 0.4496281362392621, 0.5111146759440814, 0.5111244233725083, 0.5285825317848948, 0.613328863755501, 0.6746197555468221, 0.6984568050665801, 0.7715420819139681, 0.8395018277790465, 0.8572995982151226, 0.9278042648572491]


<h1>Putting the timestamps to good use</h1>
<p>The timestamps are actually decimal values representing how far through the show the advert jingle was played. A value of 0 is at the start and 1 is at the end and they are very accurate - over 10 decimal places.</p>
<p>We loop through the array of times, sorted in chronological order.</p>
<p>The first few lines are just for debugging purposes. The first if statement checks whether the current value is the last value in the array. If it is, we end the loop.</p>
<p>The next if statement checks if the difference between the current values and the next value in the array is greater than 8 minutes. If it is, we assume this is a segment of James O'Brien as the ads breaks are always shorter than 8 mins.</p>
<p>Assuming it is, we print 'ok' and calculate the timestamp of the end of the segment. We select the section from the original 3 hour show called 'clip' in between the current timestamp and the next timestamp and add this to the end of the fin_arr.</p>

<p>After doing this for all segments longer than 8 mins, we have a 'fin_arr' with all segments longer than 8 mins.</p>

In [30]:
def calc_timeval(val):
    hours, mins, secs = calc_val(val)
    return (hours * 60) + mins
def calc_timestamp(val):
    return int(numpy.floor(val*len(clip)))

fin_arr = []
for idx, val in enumerate(sorted_times_arr):
    print([calc_val(val), fin_timestamps_arr[idx]])
    timestamp = calc_timestamp(val)
    print(timestamp)
    hours, mins, secs = calc_val(val)
    timeval = hours * 60 + mins
    if idx+1 == len(fin_times_arr):
        continue
    if calc_timeval(sorted_times_arr[idx+1]) - calc_timeval(val) > 8:
        print('ok')
        next_timestamp = calc_timestamp(sorted_times_arr[idx+1])
        fin_arr += list(clip[timestamp:next_timestamp])

[(0.0, 0.47038335647083546, 0.0), 13791]
622315
[(0.0, 3.8784511117412364, 0.0), 113871]
5131177
ok
[(0.0, 22.30276080298791, 0.0), 654971]
29506477
ok
[(0.0, 36.29162253893801, 0.0), 1065519]
48013694
ok
[(0.0, 49.65723546647845, 0.0), 1458297]
65696354
ok
[(1.0, 0.9187546763482057, 0.0), 1786055]
80595306
[(1.0, 4.605872198596179, 0.0), 1896823]
85473350
ok
[(1.0, 20.931040555932753, 0.0), 2376188]
107071493
[(1.0, 20.933064523067173, 0.0), 2376723]
107074170
ok
[(1.0, 32.00064166993466, 0.0), 2703862]
121716538
[(1.0, 32.00239620705149, 0.0), 2703982]
121718859
[(1.0, 35.144855721281054, 0.0), 2793445]
125876322
ok
[(1.0, 50.39919547599018, 0.0), 3242122]
146057762
ok
[(2.0, 1.4315559984279602, 0.0), 3560208]
160653538
[(2.0, 5.722224911984419, 0.0), 3691194]
166330078
ok
[(2.0, 18.877574744514256, 0.0), 4078454]
183734562
ok
[(2.0, 31.110329000228365, 0.0), 4440982]
199918454
[(2.0, 34.31392767872207, 0.0), 4530644]
204156805
ok
[(2.0, 47.00476767430484, 0.0), 4903246]
220946743


<p>We then save the 'fin_arr' to a file called 'test.wav' which is the 3 hour show with the adverts taken out - using the soundrate we saved right at the start!</p>

In [31]:
sf.write('test.wav', fin_arr, sr)
print(len(clip))

238139392
